# Mount drive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/') 

# Imports, Constants and Functions

## Constants

In [2]:
#@markdown Dataset
#@markdown ---
IMG_DIMS = 72 #@param {type:"slider", min:16, max:256, step:1}
NUM_CHANS = 3 #@param {type:"slider", min:1, max:4, step:1}
#number maximum of images used from the dataset (For small RAMs...)
MAX_DATA_SIZE = 7000 #@param {type:"slider", min:4000, max:35000, step:1000}
NUM_CLASSES = 8 #@param {type:"slider", min:2, max:20, step:1}
SPLIT_TRAIN_TEST = True #@param {type:"boolean"}
TRAIN_RATIO = 95 #@param {type:"slider", min:60, max:95, step:5}
#@markdown ---

#@markdown Resizing
#@markdown ---
SAVE_RESIZED = True #@param {type:"boolean"}
RESIZED_DIR = "/content/natural-images" #@param {type:"string"}
RESIZED_X=RESIZED_DIR + "train_x_{}_{}.npy".format(IMG_DIMS, MAX_DATA_SIZE)
RESIZED_Y=RESIZED_DIR + "train_y_{}_{}.npy".format(IMG_DIMS, MAX_DATA_SIZE)
RESIZED_PATH={
    'x': RESIZED_X,
    'y': RESIZED_Y
}
#@markdown ---


#@markdown Hyperparameters
#@markdown ---
#default: 50
NUM_EPOCHS = 50 #@param {type:"slider", min:10, max:300, step:5}
#default: 128
BATCH_SIZE = 256 #@param {type:"slider", min:32, max:512, step:32}
INIT_LR = 0.001 #@param {type:"number"}
#@markdown ---

#@markdown Benchmarks, tests and Outputs
#@markdown --
IMG_DIR = "/content/natural-images/data" #@param {type:"string"}
OUTPUT_PATH = "/content/gdrive/MyDrive/Colab Files/vision transformer/classification/natural-images/" #@param {type:"string"}
#@markdown ---


#@markdown Transformer Parameters
#@markdown --
patch_size = 6 #@param {type:"slider", min:2, max:20, step:1} # Size of the patches to be extract from the input images
projection_dim = 64 #@param {type:"slider", min:16, max:96, step:16}
num_heads = 4 #@param {type:"slider", min:1, max:10, step:1}
transformer_layers = 8 #@param {type:"slider", min:4, max:16, step:1}

# weight_decay = 0.0001
num_patches = (IMG_DIMS // patch_size) ** 2
transformer_units = [
    projection_dim * 2,
    projection_dim,
]  # Size of the transformer layers
mlp_head_units = [2048, 1024]  # Size of the dense layers of the final classifier

## Imports

In [3]:
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Conv2DTranspose
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Reshape
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Lambda
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
K.clear_session()
import numpy as np

import matplotlib
matplotlib.use("Agg")

from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
import cv2

import os
import random
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import MultiHeadAttention
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import LayerNormalization
from tensorflow.keras.layers import Add
from tensorflow.keras import Model
from sklearn.preprocessing import LabelEncoder
import tensorflow.keras

import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_addons as tfa

## Custom Layers

In [4]:
def mlp(x, hidden_units, dropout_rate):
    for units in hidden_units:
        x = layers.Dense(units, activation=tf.nn.gelu)(x)
        x = layers.Dropout(dropout_rate)(x)
    return x

In [5]:
class Patches(layers.Layer):
    def __init__(self, patch_size):
        super(Patches, self).__init__()
        self.patch_size = patch_size

    def call(self, images):
        batch_size = tf.shape(images)[0]
        patches = tf.image.extract_patches(
            images=images,
            sizes=[1, self.patch_size, self.patch_size, 1],
            strides=[1, self.patch_size, self.patch_size, 1],
            rates=[1, 1, 1, 1],
            padding="VALID",
        )
        patch_dims = patches.shape[-1]
        patches = tf.reshape(patches, [batch_size, -1, patch_dims])
        return patches


In [6]:
class PatchEncoder(layers.Layer):
    def __init__(self, num_patches, projection_dim):
        super(PatchEncoder, self).__init__()
        self.num_patches = num_patches
        self.projection = Dense(units=projection_dim)
        self.position_embedding = Embedding(
            input_dim=num_patches, output_dim=projection_dim
        )

    def call(self, patch):
        positions = tf.range(start=0, limit=self.num_patches, delta=1)
        encoded = self.projection(patch) + self.position_embedding(positions)
        return encoded


## Prepare Kaggle-cli

In [7]:
%%shell

#!/bin/bash
if [ -e "~/.kaggle/kaggle.json" ]
then
	echo "kaggle.json already exists"
	echo "Skipping to next cell..."
else
	echo "Preparing Kaggle-cli..."
	pip install -q kaggle
	pip install -q kaggle-cli

	mkdir -p ~/.kaggle
	cp "/content/gdrive/MyDrive/Colab Files/Kaggle/kaggle.json" ~/.kaggle/
	cat ~/.kaggle/kaggle.json 
	chmod 600 ~/.kaggle/kaggle.json
fi

Preparing Kaggle-cli...
     |████████████████████████████████| 81kB 5.6MB/s 
     |████████████████████████████████| 5.3MB 9.6MB/s 
     |████████████████████████████████| 51kB 8.5MB/s 
     |████████████████████████████████| 112kB 52.2MB/s 
     |████████████████████████████████| 143kB 61.8MB/s 
{"username":"ihabtaleb","key":"59d8093446387a51abb5f7e82b9dd58e"}

# Data Preprocessing

## Download and Unzip The DataSet



In [ ]:
%%shell

#!/bin/bash
if [ -e "/content/natural-images.zip" ]
then
		echo "Dataset already downloaded"
		echo "Skipping to next cell..."
else
		echo "Downloading Dataset..."
    kaggle datasets download -d prasunroy/natural-images
		echo "Unzipping Dataset..."
		unzip "/content/natural-images.zip" -d "/content/natural-images"
    echo "Done."
fi

## Prepare and Save The DataSet

In [9]:
def prepare_dataset(ds_exists = False, img_dim = IMG_DIMS, save_resized = SAVE_RESIZED, resized_path = RESIZED_PATH,
                    max_data_size = MAX_DATA_SIZE, split_train_test = SPLIT_TRAIN_TEST, train_ratio = TRAIN_RATIO):
  if ds_exists:
    totalImages = np.load(resized_path['x'])
    totalY = np.load(resized_path['y'])
    if split_train_test:
      train_test=int(max_data_size*(train_ratio/100))
      print ("train/test = ", train_test, "/", totalImages.shape[0]-train_test)
      return (totalImages[:train_test,:,:,:], totalY[:train_test]), (totalImages[train_test:,:,:,:], totalY[train_test:])
    else:
      return (totalImages, totalY), (np.array(), np.array())
  else:
    images=[]
    y=[]
    length=1
    dim=0
    all_img_paths = [os.path.join(dp, f) for dp, dn, fn in os.walk(IMG_DIR) for f in fn]
    random.shuffle(all_img_paths)
    for image_path in all_img_paths:
      img = cv2.imread(image_path)
      resized = cv2.resize(img,(img_dim,img_dim))
      images.append(resized)
      y.append(os.path.dirname(image_path))
      length += 1
      if length>max_data_size:
        break
    del all_img_paths
    totalImages=np.array(images)
    y_array=np.array(y)
    label_encoder = LabelEncoder()
    y_enc = label_encoder.fit_transform(y_array)
    totalY = y_enc
    del images
    totalImages = totalImages.astype("float")/255
    if save_resized:
      np.save(resized_path['x'],totalImages)
      np.save(resized_path['y'],totalY)
      print("Resized images saved in path: ", RESIZED_PATH)
    if split_train_test:
      train_test=int(max_data_size*(train_ratio/100))
      print ("train/test = ", train_test, "/", max_data_size-train_test)
      return (totalImages[:train_test,:,:,:], totalY[:train_test]), (totalImages[train_test:,:,:,:], totalY[train_test:])
    else:
      return (totalImages, totalY), (np.array(), np.array())

In [ ]:
if os.path.isfile(RESIZED_PATH['y']):
	print(".npy file already exists")
	print("Reading the existing file...")
	(x_train, y_train), (x_test, y_test) = prepare_dataset(ds_exists = True)
else:
	(x_train, y_train), (x_test, y_test) = prepare_dataset()
	print("Dataset prepared")

print(f"x_train shape: {x_train.shape} - y_train shape: {y_train.shape}")
print(f"x_test shape: {x_test.shape} - y_test shape: {y_test.shape}")

# Transformer Model

In [11]:
def create_vit_classifier():
    inputs = Input((IMG_DIMS, IMG_DIMS, NUM_CHANS))
    # Create patches.
    patches = Patches(patch_size)(inputs)
    # Encode patches.
    encoded_patches = PatchEncoder(num_patches, projection_dim)(patches)

    # Create multiple layers of the Transformer block.
    for _ in range(transformer_layers):
        attention_output = MultiHeadAttention(
            num_heads=num_heads, key_dim=projection_dim, dropout=0.1
        )(encoded_patches, encoded_patches)
        x2 = layers.Add()([attention_output, encoded_patches])
        x3 = LayerNormalization(epsilon=1e-6)(x2)
        ff = mlp(x3, hidden_units=transformer_units, dropout_rate=0.1)
        encoded_patches = Add()([x3, ff])
        encoded_patches = LayerNormalization(epsilon=1e-6)(encoded_patches)

    # Create a [batch_size, projection_dim] tensor.
    representation = Flatten()(encoded_patches)
    representation = Dropout(0.5)(representation)
    features = mlp(representation, hidden_units=mlp_head_units, dropout_rate=0.5)
    # Classify outputs.
    logits = Dense(NUM_CLASSES)(features)
    # Create the Keras model.
    model = Model(inputs=inputs, outputs=logits)
    return model


# Training

In [12]:
transformer = create_vit_classifier()

transformer.compile(
    optimizer=Adam(lr=INIT_LR),
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[
             keras.metrics.SparseCategoricalAccuracy(name="accuracy")#,
    ],
)
history = transformer.fit(x=x_train, y=y_train, batch_size=BATCH_SIZE, epochs=NUM_EPOCHS, shuffle=True, validation_split=0.1,)
print("\nEvaluating System...")
_, accuracy = transformer.evaluate(x_test, y_test)
print("Accuracy: ", round(accuracy * 100, 2), "%")

Epoch 1/50
24/24 [==============================] - 19s 417ms/step - loss: 7.4683 - accuracy: 0.1331 - val_loss: 2.0716 - val_accuracy: 0.1429
Epoch 2/50
24/24 [==============================] - 9s 370ms/step - loss: 2.1263 - accuracy: 0.1435 - val_loss: 2.0693 - val_accuracy: 0.1263
Epoch 3/50
24/24 [==============================] - 9s 372ms/step - loss: 2.0734 - accuracy: 0.1450 - val_loss: 1.9688 - val_accuracy: 0.2737
Epoch 4/50
24/24 [==============================] - 9s 374ms/step - loss: 2.0389 - accuracy: 0.1798 - val_loss: 1.9103 - val_accuracy: 0.2722
Epoch 5/50
24/24 [==============================] - 9s 375ms/step - loss: 1.9579 - accuracy: 0.2230 - val_loss: 1.8291 - val_accuracy: 0.2872
Epoch 6/50
24/24 [==============================] - 9s 377ms/step - loss: 1.8966 - accuracy: 0.2499 - val_loss: 1.7889 - val_accuracy: 0.3398
Epoch 7/50
24/24 [==============================] - 9s 379ms/step - loss: 1.8463 - accuracy: 0.2974 - val_loss: 1.7362 - val_accuracy: 0.3278
Epoch